# 1. Combinar archivos 
#### Archivo de enrgia extraida con archivo de empresas

In [1]:
import pandas as pd
import glob
import re
import os
from pathlib import Path

# === CONFIGURACIÓN ===
CENTRALES_FILE = './data/empresas_generadoras.xlsx'
INPUT_PATTERN = './downloads/extracted_precios_c_iny_*.xlsx'
OUTPUT_PREFIX = './pre_data/precios_centrales_'

# === FUNCIONES AUXILIARES ===

def detectar_fila_encabezado(df):
    for i, row in df.iterrows():
        if any(str(cell).strip().upper() == "CENTRAL" for cell in row):
            return i
    return 0

def normalizar_nombre(nombre, nombres_centrales, alias):
    x = str(nombre).strip()
    if x in alias:
        return alias[x]
    
    # Limpieza más profunda para coincidencias flexibles
    x_clean = re.sub(r'\W+', '', x).upper()
    
    for k in nombres_centrales:
        k_clean = re.sub(r'\W+', '', k).upper()
        if k_clean == x_clean:
            return k
    
    # Casos especiales para Aguaí
    if "AGUAI" in x_clean or "AGUAÍ" in x_clean:
        if "AUTOPRODUCTOR" in x_clean:
            return "Aguai (Autoproductor)"
        return "Aguaí Energia"
    
    return x

# === MAPA DE ALIAS DE CENTRALES ===
alias = {
    "Kanata en Arocagua": "Kanata ARO",
    "Kanata en Valle Hermoso": "Kanata VHE",
    "Misicuni en Arocagua": "Misicuni ARO",
    "Misicuni en Valle Hermoso": "Misicuni VHE",
    "Yunchara": "Yunchara",
    "Aguaí Energía": "Aguaí Energia",
    "AGUAÍ ENERGÍA S.A.": "Aguaí Energia",
    "Santa Cruz (Aguaí)": "Santa Cruz (Aguaí)",
    "RÍO ELÉCTRICO S.A.": "RIO ELECTRICO S.A.",
    "CHACO ENERGÍAS S.A.": "CHACO ENERGIAS S.A.",
    "RIOELEC S.A.": "RIO ELECTRICO S.A.",
}

# === PROCESAMIENTO DE ARCHIVOS ===

def procesar_archivos():
    try:
        df_centrales = pd.read_excel(CENTRALES_FILE)
        df_centrales['CENTRAL'] = df_centrales['CENTRAL'].astype(str).str.strip()

        if not {'CENTRAL', 'GENERADOR', 'TECNOLOGIA'}.issubset(df_centrales.columns):
            print("Error: El archivo debe contener columnas 'CENTRAL', 'GENERADOR' y 'TECNOLOGIA'")
            return

        mapeo_generadores = dict(zip(df_centrales['CENTRAL'], df_centrales['GENERADOR']))
        mapeo_tecnologia = dict(zip(df_centrales['CENTRAL'], df_centrales['TECNOLOGIA']))
        nombres_centrales = set(df_centrales['CENTRAL'])

        # Asegurar centrales de Aguaí en el mapeo
        for central_aguai in ["Aguaí Energia", "Aguai (Autoproductor)"]:
            if central_aguai not in mapeo_generadores:
                mapeo_generadores[central_aguai] = "AGUAÍ ENERGÍA S.A."
                mapeo_tecnologia[central_aguai] = "Biomasa"

    except Exception as e:
        print(f"Error al cargar archivo de mapeo {CENTRALES_FILE}: {str(e)}")
        return

    for input_file in glob.glob(INPUT_PATTERN):
        file_number = re.search(r'extracted_precios_c_iny_(\d+)\.xlsx', input_file)

        if not file_number:
            continue

        file_number = file_number.group(1)
        output_file = f"{OUTPUT_PREFIX}{file_number}.xlsx"

        if os.path.exists(output_file):
            print(f"[Omitido] {output_file} ya existe.")
            continue

        try:
            df_raw = pd.read_excel(input_file, header=None)
            start_row = detectar_fila_encabezado(df_raw)
            df = pd.read_excel(input_file, skiprows=start_row)
            df.columns = [str(col).strip() for col in df.columns]

            central_col = next((c for c in df.columns if 'central' in c.lower() or 'agente' in c.lower()), None)
            if central_col and central_col != 'CENTRAL':
                df = df.rename(columns={central_col: 'CENTRAL'})
            if 'CENTRAL' not in df.columns:
                print(f"[Error] No se encontró columna 'CENTRAL' en {input_file}")
                continue

            df['CENTRAL'] = df['CENTRAL'].astype(str).str.strip()

            # Eliminación de filas basura (incluyendo totales de Aguaí)
            pattern_basura = r'TOTAL|TOTALES|Nota|Tipo de cambio|nan|CARGOS POR INYECCIONES|TOTAL\s*-\s*AGUAI'
            df = df[~df['CENTRAL'].str.contains(pattern_basura, case=False, na=True, regex=True)]
            df = df[~df['CENTRAL'].str.match(r'^\d{4}-\d{2}-\d{2}', na=False)]
            df = df[~df['CENTRAL'].str.upper().str.contains(r'CENTRAL\s*ENERGIA|POTENCIA', na=False)]
            df = df[df['CENTRAL'].notna() & (df['CENTRAL'] != '')]

            # Identificación de filas válidas
            df['CENTRAL_CLEAN'] = df['CENTRAL'].str.strip().str.upper()
            centrales_validas = set(x.upper() for x in nombres_centrales)
            first_valid_idx = df[df['CENTRAL_CLEAN'].isin(centrales_validas)].index.min()
            if pd.isna(first_valid_idx):
                print(f"[Error] No se encontraron centrales válidas en {input_file}")
                continue
            df = df.loc[first_valid_idx:].copy()

            # Normalización y mapeo
            df['CENTRAL_NORMALIZADA'] = df['CENTRAL'].apply(
                lambda x: normalizar_nombre(x, nombres_centrales, alias)
            )
            df['GENERADOR'] = df['CENTRAL_NORMALIZADA'].map(mapeo_generadores)
            df['TECNOLOGIA'] = df['CENTRAL_NORMALIZADA'].map(mapeo_tecnologia)

            # Forzar presencia de ambas centrales Aguaí
            aguai_centrales = ["Aguaí Energia", "Aguai (Autoproductor)"]
            for central in aguai_centrales:
                if central not in df['CENTRAL_NORMALIZADA'].values:
                    nueva_fila = {
                        'CENTRAL': central,
                        'CENTRAL_NORMALIZADA': central,
                        'GENERADOR': "AGUAÍ ENERGÍA S.A.",
                        'TECNOLOGIA': "Biomasa"
                    }
                    for col in df.columns:
                        if 'kW' in col or 'kWh' in col:
                            nueva_fila[col] = 0.0
                    df = pd.concat([df, pd.DataFrame([nueva_fila])], ignore_index=True)

            # Procesamiento numérico
            for col in df.columns:
                if 'kW' in col or 'kWh' in col:
                    df[col] = (
                        df[col]
                        .astype(str)
                        .str.replace(',', '')
                        .str.replace(' ', '')
                        .replace('nan', None)
                        .astype(float)
                    )

            # ... código existente ...

            columnas_finales = ['CENTRAL_NORMALIZADA', 'GENERADOR', 'TECNOLOGIA'] + [
                c for c in df.columns if c not in ['CENTRAL', 'CENTRAL_NORMALIZADA', 'CENTRAL_CLEAN', 'GENERADOR', 'TECNOLOGIA']
            ]
            df_final = df[columnas_finales].rename(columns={'CENTRAL_NORMALIZADA': 'CENTRAL'})

            # === RENOMBRAR COLUMNAS ESPECÍFICAS ===
            rename_columns = {
                'Unnamed: 1': 'Precio Energía USD/MWh',
                'Unnamed: 2': 'Precio Potencia USD/kW',
                'Energía': 'Precio Energía USD/MWh',
                'Potencia Firme Remunerada': 'Precio Potencia USD/kW'
            }
            df_final = df_final.rename(columns=rename_columns)


# ... resto del código existente ...
            # Guardar el DataFrame final
            df_final.to_excel(output_file, index=False)

            print(f"[OK] {input_file} → {output_file}")

            # Detectar centrales sin mapeo
            faltantes = df_final[df_final['GENERADOR'].isna()]['CENTRAL'].unique()
            if len(faltantes) > 0:
                print(f"  ⚠️ {len(faltantes)} centrales sin GENERADOR: {faltantes[:3]}{'...' if len(faltantes) > 3 else ''}")

        except Exception as e:
            print(f"[Error] {input_file}: {str(e)}")
            try:
                Path('./errors').mkdir(exist_ok=True)
                df.to_excel(f"./errors/ERROR_{file_number}.xlsx", index=False)
            except Exception as inner_e:
                print(f"Error al guardar archivo de error: {inner_e}")

# === EJECUCIÓN PRINCIPAL ===
if __name__ == "__main__":
    print("🔄 Iniciando procesamiento de archivos...")
    procesar_archivos()
    print("✅ Proceso completado.")

🔄 Iniciando procesamiento de archivos...
[OK] ./downloads\extracted_precios_c_iny_0123.xlsx → ./pre_data/precios_centrales_0123.xlsx
[OK] ./downloads\extracted_precios_c_iny_0124.xlsx → ./pre_data/precios_centrales_0124.xlsx
  ⚠️ 1 centrales sin GENERADOR: ['Yunchará']
[OK] ./downloads\extracted_precios_c_iny_0125.xlsx → ./pre_data/precios_centrales_0125.xlsx
[OK] ./downloads\extracted_precios_c_iny_0223.xlsx → ./pre_data/precios_centrales_0223.xlsx
[OK] ./downloads\extracted_precios_c_iny_0224.xlsx → ./pre_data/precios_centrales_0224.xlsx
  ⚠️ 1 centrales sin GENERADOR: ['Yunchará']
[OK] ./downloads\extracted_precios_c_iny_0225.xlsx → ./pre_data/precios_centrales_0225.xlsx
[OK] ./downloads\extracted_precios_c_iny_0323.xlsx → ./pre_data/precios_centrales_0323.xlsx
  ⚠️ 5 centrales sin GENERADOR: ['San Borja' 'Rurrenabaque' 'Yucumo']...
[OK] ./downloads\extracted_precios_c_iny_0324.xlsx → ./pre_data/precios_centrales_0324.xlsx
[OK] ./downloads\extracted_precios_c_iny_0325.xlsx → ./pre_d

# 2. Merge archivos energia

In [2]:
import pandas as pd
import glob
from datetime import datetime

def consolidar_en_formato_largo():
    archivos = sorted(glob.glob('./pre_data/precios_centrales_*.xlsx'))
    if not archivos:
        print("No se encontraron archivos.")
        return

    registros = []

    for archivo in archivos:
        try:
            periodo = archivo.split('_')[-1].split('.')[0]
            mes = int(periodo[:2])
            año = 2000 + int(periodo[2:])
            fecha = datetime(año, mes, 1)

            df = pd.read_excel(archivo)
            if 'CENTRAL' not in df.columns:
                print(f"Omitido: {archivo} no tiene columna CENTRAL.")
                continue

            for col in df.columns:
                if col in ['CENTRAL', "GENERADOR", 'TECNOLOGIA']:
                    continue
                temp = df[['CENTRAL', "GENERADOR", 'TECNOLOGIA', col]].copy()
                temp['FECHA'] = fecha
                temp['VARIABLE'] = col
                temp = temp.rename(columns={col: 'VALOR'})
                registros.append(temp)

        except Exception as e:
            print(f"Error procesando {archivo}: {e}")
            continue

    if not registros:
        print("No se pudo consolidar ningún archivo válido.")
        return

    df_largo = pd.concat(registros, ignore_index=True)
    df_largo = df_largo[['FECHA', 'CENTRAL', "GENERADOR", 'TECNOLOGIA', 'VARIABLE', 'VALOR']]

    df_largo.to_excel("./preprocess/serie_temporal_precios.xlsx", index=False)
    print("Consolidación completada en formato largo.")
    print(f"Filas totales: {len(df_largo)}")
    print(f"Archivo guardado como 'serie_temporal_precios.xlsx'")

if __name__ == "__main__":
    consolidar_en_formato_largo()

Consolidación completada en formato largo.
Filas totales: 2534
Archivo guardado como 'serie_temporal_precios.xlsx'


# 3. Creacion de serie de tiempo de energia

In [3]:
import pandas as pd

# Leer archivo
df = pd.read_excel("./preprocess/serie_temporal_precios.xlsx")

# Convertir FECHA y eliminar filas sin fecha válida
df['FECHA'] = pd.to_datetime(df['FECHA'], errors='coerce', unit='d')
df = df.dropna(subset=['FECHA'])

# Limpiar espacios en columnas clave
df['CENTRAL'] = df['CENTRAL'].astype(str).str.strip()
df['TECNOLOGIA'] = df['TECNOLOGIA'].astype(str).str.strip()
df['VARIABLE'] = df['VARIABLE'].astype(str).str.strip()

# Filtrar sólo las variables de interés exactas (con mayúsculas y espacios)
df = df[df['VARIABLE'].isin(['Precio Energía USD/MWh', 'Precio Potencia USD/kW'])]

# Asegurar que VALOR sea numérico
df['VALOR'] = pd.to_numeric(df['VALOR'], errors='coerce')
df = df.dropna(subset=['VALOR'])

# Crear MES_ANIO
df['MES_ANIO'] = df['FECHA'].dt.strftime('%m%Y')

# Reemplazar valores faltantes en AGENTE y EMPRESA con forward fill
df['CENTRAL'] = df['CENTRAL'].replace('nan', pd.NA).fillna(method='ffill')
df['TECNOLOGIA'] = df['TECNOLOGIA'].replace('nan', pd.NA).fillna(method='ffill')

# Crear columna pivote combinando variable y mes_año (con espacios y mayúsculas)
df['COLUMNA'] = df['VARIABLE'] + ' ' + df['MES_ANIO']

# Pivotear el DataFrame
df_pivot = df.pivot_table(index=['CENTRAL', 'TECNOLOGIA'], columns='COLUMNA', values='VALOR', aggfunc='first')

# Función para extraer año y mes para ordenar correctamente
def extraer_anio_mes(col_name):
    fecha = col_name.split()[-1]  # '012023'
    mes = int(fecha[:2])
    anio = int(fecha[2:])
    return (anio, mes)

# Obtener lista de columnas
cols = df_pivot.columns.tolist()

# Separar en Energía y Potencia
energia_cols = sorted([c for c in cols if c.startswith('Precio Energía USD/MWh')], key=extraer_anio_mes)
potencia_cols = sorted([c for c in cols if c.startswith('Precio Potencia USD/kW')], key=extraer_anio_mes)

# Concatenar para orden final
cols_ordenadas = energia_cols + potencia_cols

# Reordenar columnas
df_pivot = df_pivot[cols_ordenadas]

# Resetear índice
df_final = df_pivot.reset_index()

# Guardar resultado
df_final.to_excel("./preprocess/serie_precios_cronologica.xlsx", index=False)


C:\Users\daniel.canedo\AppData\Local\Temp\ipykernel_22428\486268622.py:26: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['CENTRAL'] = df['CENTRAL'].replace('nan', pd.NA).fillna(method='ffill')
C:\Users\daniel.canedo\AppData\Local\Temp\ipykernel_22428\486268622.py:27: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['TECNOLOGIA'] = df['TECNOLOGIA'].replace('nan', pd.NA).fillna(method='ffill')


# 4. Depuracion de serie de tiempo de energia

In [4]:
import pandas as pd

# Leer el archivo Excel
df = pd.read_excel("./preprocess/serie_precios_cronologica.xlsx")

# Lista de agentes a eliminar (como lista plana)
agentes_a_eliminar = [
    "TOTAL - CESSA", "TOTAL - CRE R.L.", "TOTAL CRE", "TOTAL - DELAPAZ", 
    "TOTAL - ELFEC", "TOTAL - ENDE", "TOTAL ENDE DELBENI S.A.M.",
    "TOTAL - ENDE DEORURO S.A.", "TOTALES", "Tipo de cambio",
    "TOTAL - SEPSA", "TOTAL - SETAR"
]

# Filtrar y eliminar las filas no deseadas
df = df[~df["CENTRAL"].isin(agentes_a_eliminar)]

# Filtrar solo columnas con precios de energía
columnas_energia = [col for col in df_final.columns if col.startswith("Precio Energía USD/MWh")]
df_energia = df_final[['CENTRAL', 'TECNOLOGIA'] + columnas_energia]

# Guardar nuevo archivo solo con precios de energía
df_energia.to_excel("./data/serie_precios_energia.xlsx", index=False)


In [5]:
import pandas as pd

# Leer el archivo Excel
df = pd.read_excel("./preprocess/serie_precios_cronologica.xlsx")

# Lista de agentes a eliminar (como lista plana)
agentes_a_eliminar = [
    "TOTAL - CESSA", "TOTAL - CRE R.L.", "TOTAL CRE", "TOTAL - DELAPAZ", 
    "TOTAL - ELFEC", "TOTAL - ENDE", "TOTAL ENDE DELBENI S.A.M.",
    "TOTAL - ENDE DEORURO S.A.", "TOTALES", "Tipo de cambio",
    "TOTAL - SEPSA", "TOTAL - SETAR"
]

# Filtrar y eliminar las filas no deseadas
df = df[~df["CENTRAL"].isin(agentes_a_eliminar)]

# Filtrar solo columnas con precios de energía
columnas_energia = [col for col in df_final.columns if col.startswith("Precio Potencia USD/kW")]
df_energia = df_final[['CENTRAL', 'TECNOLOGIA'] + columnas_energia]

# Guardar nuevo archivo solo con precios de energía
df_energia.to_excel("./data/serie_precios_potencia.xlsx", index=False)